In [1]:
import numpy as np
import pandas as pd

In [3]:
y=pd.read_csv("Data/Dataset_labels_scaled_max.csv",index_col=0)
X=np.load('Data/Latent_space.npy')

In [4]:
y.head()

,IDENT,Manual_noise_sigma,Original_size,fit_dvc_btt,fit_mad_b,fit_mad_s,flux,flux_radius,mag_auto,maxI_4_CV,...,HLRcirc_Bulge,HLRcirc_Disk,fit_status_0,fit_status_1,fit_status_2,fit_status_3,fit_status_4,SNR_Manual,NonUniformity,maxI
0,141190,0.008710,242,0.072256,0.002850,0.002923,99.841270,7.310604,21.040642,0.441848,...,0.00000,0.000000,1.0,1.0,1.0,1.0,1.0,50.727741,42.000836,1.023790
1,143346,0.007900,144,0.006207,0.002618,0.002648,47.668005,8.154056,21.782894,0.184937,...,0.00000,0.000000,1.0,1.0,1.0,1.0,1.0,23.411050,29.116470,0.345019
2,93315,0.005307,92,0.008065,0.004632,0.004639,21.396968,4.657753,22.861856,0.149750,...,0.00000,0.000000,1.0,1.0,1.0,1.0,1.0,28.217329,61.255597,0.273879
3,129658,0.005357,112,0.191620,0.002874,0.002959,24.274229,10.014692,22.504494,0.108650,...,8.92241,7.938353,1.0,1.0,1.0,1.0,1.0,20.282247,26.478137,0.150976
4,9205,0.007133,124,0.050223,0.002835,0.002975,37.317560,7.358890,22.131260,0.153872,...,0.00000,0.000000,1.0,1.0,1.0,1.0,1.0,21.572091,27.345317,0.250405


In [9]:
y.columns[y.columns.str.startswith('Sersic')]

Index(['Sersic_I', 'Sersic_HLR', 'Sersic_n', 'Sersic_q', 'Sersic_boxiness',
       'Sersic_phi'],
      dtype='object')

In [10]:
y_Sersic=y[['Sersic_HLR', 'Sersic_n', 'Sersic_q','Sersic_phi']]

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y_Sersic, test_size=0.2)

In [28]:
def test_model(model,x,y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    models={}
    for i,column in enumerate(y.columns):
        regr=model(random_state=42)
        regr.fit(x_train, y_train[column])
        models[column]=regr
        print(column,r2_score(y_test[column],regr.predict(x_test)))
    return models

In [13]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=42)
regr.fit(x_train, y_train)

RandomForestRegressor(random_state=42)

In [14]:
from sklearn.metrics import r2_score
r2_score(y_test,regr.predict(x_test))

0.550718711643557

In [34]:
Forests=test_model(RandomForestRegressor,X,y_Sersic)

Sersic_HLR 0.6743067671809722
Sersic_n 0.7970699129349867
Sersic_q 0.7793512071879496
Sersic_phi 0.2847322939656446


In [35]:
from sklearn.ensemble import GradientBoostingRegressor
GrBoosts=test_model(GradientBoostingRegressor,X,y_Sersic)

Sersic_HLR 0.682893654861875
Sersic_n 0.7960814449517473
Sersic_q 0.6981602034518656
Sersic_phi 0.24780566117176028


In [33]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
grB=MultiOutputRegressor(
    GradientBoostingRegressor(random_state=42))
grB.fit(x_train, y_train)
r2_score(y_test,grB.predict(x_test))

0.5987481378441584